In [1]:
import requests
import pandas as pd
import matplotlib
import plotly.express as px

In [62]:
API_Key = 'A05Bru6X2w08Us4SI3Et2tN4SP9ZebZJ7B8OYfXhMrr'

In [24]:
from datetime import datetime, timedelta
end_date = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
start_date = (datetime.now()-timedelta(days=10)).strftime('%Y-%m-%d %H:%M:%S')

In [101]:
headers = {'Authorization': API_key}
params = {'start-date': start_date, 'end-date': end_date}
zone = "SE-SE3"
url = 'https://api.rebase.energy/energy/consumption/{0}/'.format(zone)
response = requests.get(url, headers=headers, params=params)
result = response.json()

In [102]:
result["records"][23]

{'fetch_time': '2023-08-25T02:12:31+00:00',
 'timestamp': '2023-08-24T15:00:00+00:00',
 'source': 'entsoe.eu',
 'consumption': 8534.0}

In [103]:
def get_consumption_data(zone, API_key): 
    headers = {'Authorization': API_key}
    params = {'start-date': start_date, 'end-date': end_date}
    
    url = 'https://api.rebase.energy/energy/consumption/{0}/'.format(zone)
    response = requests.get(url, headers=headers, params=params)
    result = response.json()

    consumption, time_consumption = [], []
    for record in result['records']:
        consumption.append(record['consumption'])
        time_consumption.append(record['timestamp'])

    df_consumption = pd.DataFrame(index=time_consumption, data={"consumption": consumption})
    df_consumption.index = pd.to_datetime(df_consumption.index)

    url = 'https://api.rebase.energy/energy/consumption-forecast/{0}/'.format(zone)
    response = requests.get(url, headers=headers, params=params)
    result = response.json()

    forecast, time_forecast = [], []
    for record in result['records']:
        forecast.append(record['value'])
        time_forecast.append(record['timestamp'])

    df_forecast = pd.DataFrame(index=time_forecast, data={"forecast": forecast})
    df_forecast.index = pd.to_datetime(df_forecast.index)

    df = pd.concat([df_consumption, df_forecast])
    
    return df

In [104]:
API_key = 'A05Bru6X2w08Us4SI3Et2tN4SP9ZebZJ7B8OYfXhMrr'
df = get_consumption_data("SE-SE3", API_key)

UnboundLocalError: cannot access local variable 'df_forecast' where it is not associated with a value

In [65]:
def get_production_data(zone, variables, API_key): 
    headers = {'Authorization': API_key}
    params = {'start-date': start_date, 'end-date': end_date}
    url = 'https://api.rebase.energy/energy/production/{0}/'.format(zone)

    response = requests.get(url, headers=headers, params=params)
    result = response.json()

    data = {variable: [] for variable in variables}
    time_stamp = []
    for record in result['records']:
        for variable in variables: 
            data[variable].append(record['production'][variable])
        time_stamp.append(record['timestamp'])

    df = pd.DataFrame(index=time_stamp, data=data)
    df.index = pd.to_datetime(df.index)

    return df

In [70]:
API_key = 'A05Bru6X2w08Us4SI3Et2tN4SP9ZebZJ7B8OYfXhMrr'
df = get_production_data("SE-SE3", ["wind", "solar"], API_key)

In [76]:
fig = px.line(df, x=df.index, y="solar", color_discrete_sequence=["yellow"])
fig.show()

/Users/sebaheg/Documents/Github/powerdash/.venv_powerdash/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [69]:
df

,wind
2023-08-23 16:00:00+00:00,212.0
2023-08-23 17:00:00+00:00,232.0
2023-08-23 18:00:00+00:00,219.0
2023-08-23 19:00:00+00:00,228.0
2023-08-23 20:00:00+00:00,232.0
...,...
2023-09-01 21:00:00+00:00,566.0
2023-09-01 22:00:00+00:00,543.0
2023-09-01 23:00:00+00:00,414.0
2023-09-02 00:00:00+00:00,295.0
